<a href="https://colab.research.google.com/github/mukul-mschauhan/Machine-Learning-Projects/blob/master/Criminal_Face_Detection_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install opencv-python
!pip install dlib
!pip install face_recognition
!pip install numpy
!pip install tkinter

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.1/100.1 MB 7.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566164 sha256=b0777a95e3e61cca6da40a580a3fa9d6088daa99fa96d783e13bf04927cdfa3f
  Stored in directory: /root/.cache/pip/wheels/7a/eb/cf/e9eced74122b679557f597bb7c8e4c739cfcac526db1fd523d
Successfully built face-recognition-models
ERROR: Could not find a version that satisfies the requirement tkinter (from versions: none)
ERROR: No matching distribution found for tkinter


### Step 2: Face Detection with OpenCV

To detect faces in an image or video stream, we will use OpenCV’s Haar Cascade Classifier. This classifier is pre-trained to detect faces and is widely used in computer vision tasks.

In [ ]:
import cv2

# Load the pre-trained face detection classifier
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Open the webcam or video file
cap = cv2.VideoCapture(0)  # 0 for the default webcam

while True:
    # Read each frame from the webcam
    ret, frame = cap.read()

    # Convert to grayscale for detection
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces in the image
    faces = face_cascade.detectMultiScale(gray, 1.1, 4)

    # Draw rectangles around each face
    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)

    # Display the output
    cv2.imshow('Face Detection', frame)

    # Break the loop on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the capture object and close all windows
cap.release()
cv2.destroyAllWindows()

### Step 3: Face Recognition with face_recognition Library

To recognize faces, we will use the face_recognition library, which provides a simple interface for facial recognition using deep learning models. It can compare faces against a database of known faces and identify if there’s a match.

In [ ]:
import face_recognition
import cv2
import numpy as np

# Load a criminal image from the database (known face)
criminal_image = face_recognition.load_image_file("criminal.jpg")
criminal_face_encoding = face_recognition.face_encodings(criminal_image)[0]

# Array of known face encodings (could be multiple criminals)
known_face_encodings = [criminal_face_encoding]
known_face_names = ["John Doe"]  # Criminal name for display

# Initialize the webcam
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()

    # Resize the frame for faster face recognition processing
    small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)

    # Convert to RGB since face_recognition uses RGB format
    rgb_small_frame = small_frame[:, :, ::-1]

    # Find all faces and their encodings in the current frame
    face_locations = face_recognition.face_locations(rgb_small_frame)
    face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)

    # Loop through each face found in the frame
    for face_encoding in face_encodings:
        # Compare the face to the known faces
        matches = face_recognition.compare_faces(known_face_encodings, face_encoding)

        name = "Unknown"

        # If a match is found, display the criminal's name
        if True in matches:
            first_match_index = matches.index(True)
            name = known_face_names[first_match_index]

        # Display the name below the face
        for (top, right, bottom, left) in face_locations:
            top *= 4
            right *= 4
            bottom *= 4
            left *= 4
            cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)
            cv2.putText(frame, name, (left + 6, bottom - 6), cv2.FONT_HERSHEY_DUPLEX, 0.75, (255, 255, 255), 1)

    # Display the resulting image
    cv2.imshow('Criminal Face Recognition', frame)

    # Break loop on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam and close windows
cap.release()
cv2.destroyAllWindows()


In this code:

The criminal face encoding is compared with the detected faces.
If a match is found, the name of the criminal is displayed on the screen.

### Step 4: Storing and Managing the Criminal Database
We will use an SQLite database or a CSV file to store criminal face encodings along with metadata such as names, crimes, etc.

In [ ]:
import sqlite3
import face_recognition

# Connect to SQLite database (or create it)
conn = sqlite3.connect('criminals.db')
cursor = conn.cursor()

# Create a table to store criminal face encodings
cursor.execute('''
CREATE TABLE IF NOT EXISTS Criminals (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL,
    face_encoding BLOB NOT NULL
)
''')

# Insert a criminal's face encoding into the database
def insert_criminal(name, image_path):
    # Load image and encode the face
    image = face_recognition.load_image_file(image_path)
    face_encoding = face_recognition.face_encodings(image)[0]

    # Store the face encoding as a binary string
    cursor.execute("INSERT INTO Criminals (name, face_encoding) VALUES (?, ?)", (name, face_encoding.tobytes()))
    conn.commit()

# Example: Add a new criminal to the database
insert_criminal("John Doe", "criminal.jpg")

# Query criminal data from the database
def get_criminals():
    cursor.execute("SELECT * FROM Criminals")
    return cursor.fetchall()

# Close the connection
conn.close()


### Step 5: Implementing an Alert System
When a match is found between the detected face and a known criminal, the system should generate an alert (e.g., print a message to the console or send an email).

In [ ]:
def alert_authorities(criminal_name):
    print(f"ALERT! Criminal {criminal_name} has been detected!")


### Step 6: Building the GUI (Optional)
You can use Tkinter to build a basic interface that allows users to upload a new criminal’s face, view the detection status, and manage the criminal database.

In [ ]:
import tkinter as tk
from tkinter import filedialog

# Initialize Tkinter window
root = tk.Tk()
root.title("Criminal Face Detection System")
root.geometry("400x400")

# Function to upload a new criminal face
def upload_new_criminal():
    file_path = filedialog.askopenfilename()
    name = input("Enter the criminal's name: ")
    insert_criminal(name, file_path)

# Button to upload new criminal
upload_button = tk.Button(root, text="Upload New Criminal", command=upload_new_criminal)
upload_button.pack(pady=20)

# Start Tkinter main loop
root.mainloop()

## 7. Full Prototype Structure
This is the structure for the Criminal Face Detection System:

Face Detection: Using OpenCV to detect faces in real-time.
Face Recognition: Using the face_recognition library to match detected faces against a criminal database.
Database: Storing criminal face encodings using SQLite for persistence.
Alert System: Sending notifications or printing alerts when a match is found.
GUI (Optional): Building a user-friendly interface for managing the system.